In [52]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [55]:
def readData(location):
  if(location == 1):
    y_train_obs_A = pd.read_parquet('A/train_targets.parquet')
    X_test_est_A = pd.read_parquet('A/X_test_estimated.parquet')
    X_train_est_A = pd.read_parquet('A/X_train_estimated.parquet')
    X_train_obs_A = pd.read_parquet('A/X_train_observed.parquet')
    return y_train_obs_A, X_test_est_A, X_train_est_A, X_train_obs_A
  elif(location == 2):
    y_train_obs_B = pd.read_parquet('B/train_targets.parquet')
    X_test_est_B = pd.read_parquet('B/X_test_estimated.parquet' )
    X_train_est_B = pd.read_parquet('B/X_train_estimated.parquet')
    X_train_obs_B = pd.read_parquet('B/X_train_observed.parquet')
    return y_train_obs_B, X_test_est_B , X_train_est_B, X_train_obs_B
  else:
    y_train_obs_C = pd.read_parquet('C/train_targets.parquet')
    X_test_est_C = pd.read_parquet('C/X_test_estimated.parquet')
    X_train_est_C = pd.read_parquet('C/X_train_estimated.parquet')
    X_train_obs_C = pd.read_parquet('C/X_train_observed.parquet')
    X_c = pd.concat([X_train_est_C, X_train_obs_C])
    return y_train_obs_C, X_test_est_C, X_train_est_C, X_train_obs_C

In [56]:
y_a, X_test_A, X_train_A_est, X_train_A_obs = readData(3)

In [57]:
X_train_A_obs.set_index('date_forecast', inplace=True)

X_train_A_est.set_index('date_forecast', inplace=True)

X_test_A.set_index('date_forecast', inplace=True)

y_a.set_index('time', inplace=True)
X_test_A.shape

(2880, 46)

In [58]:
def make_hourly(unsampled_df, target_df):
    sampled_df = unsampled_df.resample('H').mean()
    sampled_df.index = sampled_df.index.floor('H')
    merged_df = sampled_df.merge(target_df, left_index=True, right_index=True, how='inner')
    return merged_df

### A

In [59]:
merged_df_train_obs_A = make_hourly(X_train_A_obs, y_a)
# Reimposta l'indice se lo desideri
merged_df_train_est_A = make_hourly(X_train_A_est, y_a)
print(merged_df_train_obs_A.shape)
print(merged_df_train_est_A.shape)


(29200, 46)
(2954, 47)


In [60]:
X_test_A = X_test_A.fillna(0)

X_test_A = X_test_A.resample('H').mean()
X_test_A.index = X_test_A.index.floor('H')

X_test_A = X_test_A.dropna()
X_test_A

,date_calc,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,...,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms
date_forecast,,,,,,,,,,,,,,,,,,,,,
2023-05-01 00:00:00,2023-04-30 07:00:04,4.150,1.28600,0.000000,0.000000,0.000000,1236.500000,0.0,271.024994,0.000000,...,16.025501,-10.578000,0.000,273.700012,69.300003,31241.974609,3.425,1.450,3.100,0.0
2023-05-01 01:00:00,2023-04-30 07:00:04,4.050,1.28550,1107.824951,0.000000,0.000000,1220.425049,0.0,270.700012,0.000000,...,30.496750,-7.928250,0.000,273.750000,78.525002,31019.550781,3.325,1.400,3.025,0.0
2023-05-01 02:00:00,2023-04-30 07:00:04,3.900,1.28375,1477.099976,0.000000,0.000000,1258.650024,0.0,270.200012,0.000000,...,44.516251,-3.843750,0.000,273.849976,89.275002,32372.800781,3.225,1.225,3.000,0.0
2023-05-01 03:00:00,2023-04-30 07:00:04,3.750,1.28275,0.000000,9972.500000,11.549999,1655.175049,0.0,269.700012,9.300000,...,58.079498,1.391000,0.000,273.950012,57.750000,35433.625000,3.125,0.950,3.000,0.0
2023-05-01 04:00:00,2023-04-30 07:00:04,3.700,1.28150,1492.500000,141106.671875,76.675003,1639.099976,0.0,269.450012,47.924999,...,71.333252,7.454750,0.000,273.950012,60.474998,35455.574219,3.025,0.700,2.950,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-03 19:00:00,2023-07-02 07:00:31,8.400,1.19675,3493.649902,475981.375000,84.775002,1967.375000,0.0,281.700012,32.200001,...,306.688995,8.131001,0.100,286.850006,88.175003,41007.898438,2.175,1.900,-1.075,0.0
2023-07-03 20:00:00,2023-07-02 07:00:31,8.600,1.20000,3078.750000,183261.343750,24.549999,1449.500000,0.0,282.024994,13.875000,...,319.703003,3.239500,0.000,286.450012,78.949997,41315.949219,2.200,2.000,-0.925,0.0
2023-07-03 21:00:00,2023-07-02 07:00:31,8.875,1.20350,2308.399902,36831.074219,1.225000,1543.650024,0.0,282.350006,1.250000,...,333.037231,-0.477000,0.000,285.950012,80.449997,41665.898438,2.250,2.050,-0.950,0.0


### B

In [11]:
merged_df_train_obs_B = make_hourly(X_train_B_obs, y_b)
merged_df_train_est_B = make_hourly(X_train_B_est, y_b)
print(merged_df_train_obs_B.shape)
print(merged_df_train_est_B.shape)


(29222, 46)
(3625, 46)


C:\Users\claxl\AppData\Local\Temp\ipykernel_12432\3041205111.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sampled_df = unsampled_df.resample('H').mean()


In [39]:
X_test_B = X_test_B.fillna(0)

X_test_B = X_test_B.resample('H').mean()
X_test_B.index = X_test_B.index.floor('H')
X_test_B = X_test_B.dropna()

X_test_B.shape

C:\Users\claxl\AppData\Local\Temp\ipykernel_26728\1178095122.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  X_test_B = X_test_B.resample('H').mean()


(720, 45)

### C

In [13]:
merged_df_train_obs_C = make_hourly(X_train_C_obs, y_c)
merged_df_train_est_C = make_hourly(X_train_C_est, y_c)
print(merged_df_train_obs_C.shape)
print(merged_df_train_est_C.shape)


(29200, 46)
(2954, 46)


C:\Users\claxl\AppData\Local\Temp\ipykernel_12432\3041205111.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sampled_df = unsampled_df.resample('H').mean()


In [40]:
X_test_C = X_test_C.fillna(0)

X_test_C = X_test_C.resample('H').mean()
X_test_C.index = X_test_C.index.floor('H')
X_test_C = X_test_C.dropna()

X_test_C.shape

C:\Users\claxl\AppData\Local\Temp\ipykernel_26728\3087904568.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  X_test_C = X_test_C.resample('H').mean()


(720, 45)

### SAVE

In [31]:
base = ""

In [61]:
merged_df_train_obs_A.to_csv(base + 'train_obs_C.csv',index = True)
merged_df_train_est_A.to_csv(base + 'train_est_C.csv',index = True)
X_test_A.to_csv(base + 'X_test_C.csv',index = True)

In [45]:
#merged_df_train_obs_B.to_csv(base + 'B/train_obs_B.csv',index = True)
#merged_df_train_est_B.to_csv(base + 'B/train_est_B.csv',index = True)
X_test_B.to_csv(base + 'B/X_test_B.csv',index = True)

In [46]:
#merged_df_train_obs_C.to_csv(base + 'C/train_obs_C.csv',index = True)
#merged_df_train_est_C.to_csv(base + 'C/train_est_C.csv',index = True)
X_test_C.to_csv(base + 'C/X_test_C.csv',index = True)

In [43]:
merged_df_train_obs_A

,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,...,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,pv_measurement
2019-06-02 22:00:00,7.700,1.22825,1728.949951,0.000000,0.00,1728.949951,0.0,280.299988,0.000,0.000000,...,-3.774250,0.0,286.225006,100.000000,40386.476562,3.600,-3.575,-0.500,0.0,0.00
2019-06-02 23:00:00,7.700,1.22350,1689.824951,0.000000,0.00,1689.824951,0.0,280.299988,0.000,0.000000,...,-4.357250,0.0,286.899994,100.000000,33770.648438,3.350,-3.350,0.275,0.0,0.00
2019-06-03 00:00:00,7.875,1.21975,1563.224976,0.000000,0.00,1563.224976,0.0,280.649994,0.000,0.000000,...,-3.309500,0.0,286.950012,100.000000,13595.500000,3.050,-2.950,0.750,0.0,0.00
2019-06-03 01:00:00,8.425,1.21800,1283.425049,208.649994,0.75,1283.425049,0.0,281.674988,0.300,526.775024,...,-0.822500,0.0,286.750000,100.000000,2321.850098,2.725,-2.600,0.875,0.0,0.00
2019-06-03 02:00:00,8.950,1.21800,1003.500000,32468.150391,23.10,1003.500000,0.0,282.500000,11.975,22068.949219,...,3.051250,0.0,286.450012,99.224998,11634.799805,2.550,-2.350,0.925,0.0,19.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-20 20:00:00,5.800,1.28050,1291.500000,0.000000,0.00,1192.550049,0.0,275.799988,0.000,0.000000,...,-29.704250,0.0,278.250000,47.050003,40105.472656,1.925,0.475,1.850,0.0,0.00
2022-10-20 21:00:00,5.575,1.28400,NaN,0.000000,0.00,686.900024,0.0,275.200012,0.000,0.000000,...,-33.860748,0.0,277.850006,19.500000,39001.398438,1.675,0.250,1.675,0.0,0.00
2022-10-20 22:00:00,5.350,1.28700,NaN,0.000000,0.00,149.500000,0.0,274.600006,0.000,0.000000,...,-36.270000,0.0,277.475006,9.700000,35336.726562,1.400,0.050,1.400,0.0,0.00
2022-10-20 23:00:00,5.200,1.29050,NaN,0.000000,0.00,NaN,0.0,274.174988,0.000,0.000000,...,-36.614998,0.0,277.149994,4.525000,27164.625000,1.300,-0.175,1.300,0.0,0.00
